In [9]:
import sklearn.cluster as cl
import sklearn.mixture as mi
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score
import pandas as pd
import time
import torch
from torch import nn
from torch.utils.data import DataLoader
from numpy.linalg import norm
from numpy import dot
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import OneHotEncoder

In [10]:
df = pd.read_csv("DES.csv")
df

,Unnamed: 0,patient_id,geslacht,geboortedatum,opname_nummer,zkh_opn_start,herkomst,marital_status,race,diagnose,icd_version,start_operatie,endtime,duur_ok_minutes,length_of_stay_days,department,time_to_surgery_minutes,target
0,0,10000032,F,2180-09-09,22595853,2180-05-06 22:23:00,Other,WIDOWED,WHITE,07070,9,2180-05-06 19:17:00,2180-05-07 17:21:27,1324.450000,0.786111,MED,-186.0,0
1,1,10000032,F,2180-09-09,22841357,2180-06-26 18:27:00,Medicaid,WIDOWED,WHITE,07071,9,2180-06-26 15:54:00,2180-06-27 18:49:12,1615.200000,1.015278,MED,-153.0,0
2,2,10000032,F,2180-09-09,25742920,2180-08-05 23:44:00,Medicaid,WIDOWED,WHITE,07054,9,2180-08-05 20:58:00,2180-08-07 17:50:44,2692.733333,1.754167,MED,-166.0,0
3,3,10000068,F,NaN,25022803,2160-03-03 23:16:00,Other,SINGLE,WHITE,30500,9,2160-03-03 21:55:00,2160-03-04 07:33:47,578.783333,0.298611,MED,-81.0,0
4,4,10000117,F,NaN,27988844,2183-09-18 18:10:00,Other,DIVORCED,WHITE,E7800,10,2183-09-18 08:41:00,2183-09-21 16:30:30,4789.500000,2.930556,ORTHO,-569.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229416,229416,19999828,F,NaN,25744818,2149-01-08 16:44:00,Other,SINGLE,WHITE,B954,10,2149-01-08 09:11:00,2149-01-18 18:44:40,14973.666667,10.011111,MED,-453.0,1
229417,229417,19999828,F,NaN,29734428,2147-07-18 16:23:00,Other,SINGLE,WHITE,B9620,10,2147-07-17 17:18:00,2147-08-04 18:12:35,25974.583333,17.074306,SURG,-1385.0,1
229418,229418,19999840,M,2164-09-17,21033226,2164-09-10 13:47:00,Other,WIDOWED,WHITE,2724,9,2164-09-10 11:09:00,2164-09-17 16:35:15,10406.250000,6.996528,MED,-158.0,1
229419,229419,19999840,M,2164-09-17,26071774,2164-07-25 00:27:00,Other,WIDOWED,WHITE,2724,9,2164-07-24 21:16:00,2164-07-28 12:22:30,5226.500000,3.491667,NMED,-191.0,1


In [11]:
dfClean = df.drop(columns=['Unnamed: 0', "patient_id", "geboortedatum", "opname_nummer", "zkh_opn_start", "icd_version", "start_operatie", "endtime", "duur_ok_minutes", "length_of_stay_days", "time_to_surgery_minutes", "target", "diagnose"])

In [12]:
def get_similarity_list(data,target_index,
                        method = 'jaccard',
                        sort = 'descending',
                        n_items = -1,
                        unique_only = False,
                        exclude_self = True
                       ):
    """
    This function produces a list of dictionaries based on the similarity between a target item and other items in a dataset.

    :param data: a dataframe of the dataset
    :param target_index: the index of the item you want to compare all other items against
    :param method: the distance method e.g. jaccard, hamming, dice, etc. all methods can be found in the scipy spatial.distance docs: https://docs.scipy.org/doc/scipy/reference/spatial.distance.html
    :param sort: ascending, descending or None (None defaults to sorting on index)
    :param n_items: the number of items to return
    :param unique_only: True if you want to compare the target against unique items only, False if you want to compare the target against all items
    :param exclude_self: True if you want to exclude the target, False if you want to include the target
    :return: A list of dictionaries with keys: index, similarity, n_duplicates}
            index: the index of the item in the dataframe
            similarity: the similarity measure between an item and the target item
            n_duplicates: the number of items with the same columns
    """
    
    
    
    # one hot encode the data so that it works with every distance method    
    one_hot_data = OneHotEncoder(dtype = bool).fit_transform(data).toarray()
    
    similarity_df = data.copy()
    
    # 1- the distance score, is the similarity score
    similarity_df['similarity'] = list(1-pairwise_distances(one_hot_data, Y = np.array(one_hot_data[target_index]).reshape(-1,1).T, metric = method)[:,0])
    
    #calculate the number of duplicates for every item in the data
    similarity_df['n_duplicates'] = data.groupby(list(data.columns), sort=False)[(list(data.columns))[0]].transform('size') - 1
    
    similarity_df['index'] = similarity_df.index
    
    if unique_only:
        similarity_df = similarity_df.drop_duplicates(subset = list(similarity_df.drop(columns = ['index','similarity','n_duplicates']).columns))
        
    if exclude_self:
        similarity_df = similarity_df.drop(index=similarity_df.iloc[target_index].name)
        
    similarity_list = similarity_df[['index','similarity','n_duplicates']].to_dict('records')
    
    if sort == 'descending':
        sort_direction = -1
    elif sort == 'ascending':
        sort_direction = 1
    else:
        return similarity_list[:n_items]
        
    return sorted(similarity_list, key=lambda d: d['similarity'])[::sort_direction][:n_items]
        

In [13]:
get_similarity_list(data = dfClean,
                    target_index = 0,
                    method = 'jaccard',
                    sort = "descending",
                    n_items = 10,
                    unique_only = True,
                    exclude_self = True
                   )

[{'index': 202812, 'similarity': 0.6666666666666667, 'n_duplicates': 1},
 {'index': 136394, 'similarity': 0.6666666666666667, 'n_duplicates': 5},
 {'index': 110493, 'similarity': 0.6666666666666667, 'n_duplicates': 3},
 {'index': 103416, 'similarity': 0.6666666666666667, 'n_duplicates': 0},
 {'index': 96734, 'similarity': 0.6666666666666667, 'n_duplicates': 2},
 {'index': 81782, 'similarity': 0.6666666666666667, 'n_duplicates': 3},
 {'index': 71180, 'similarity': 0.6666666666666667, 'n_duplicates': 1},
 {'index': 67034, 'similarity': 0.6666666666666667, 'n_duplicates': 0},
 {'index': 64980, 'similarity': 0.6666666666666667, 'n_duplicates': 4},
 {'index': 63125, 'similarity': 0.6666666666666667, 'n_duplicates': 4}]

In [14]:
dfClean.loc[0]

geslacht                F
herkomst            Other
marital_status    WIDOWED
race                WHITE
department            MED
Name: 0, dtype: object

In [15]:
dfClean.loc[1]

geslacht                 F
herkomst          Medicaid
marital_status     WIDOWED
race                 WHITE
department             MED
Name: 1, dtype: object